<a href="https://colab.research.google.com/github/parmpatel1494/PageRank/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
#link = 'https://drive.google.com/file/d/1N5W93beC_PrXsSFO5bjPVqISKQO9ISG6/view?usp=sharing'
link = 'https://drive.google.com/file/d/1hPAwrAzKvxS33f0kmVBB5uxTmL-XeRVK/view?usp=sharing'
import pandas as pd

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('sentimentdata.csv')

df = pd.read_csv('sentimentdata.csv')
print(df)


UnicodeDecodeError: ignored

TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [ ]:
df.head

<bound method NDFrame.head of                                         SentimentText  Sentiment
0   first think another Disney movie, might good, ...          1
1   Put aside Dr. House repeat missed, Desperate H...          0
2   big fan Stephen King's work, film made even gr...          1
3   <br>watched horrid thing TV. Needless say one ...          0
4   truly enjoyed film. acting terrific plot. Jeff...          1
5   memory "The Last Hunt" stuck since saw 1956 13...          1
6   Shakespeare fan, appreciate Ken Branagh done b...          0
7   privilege watching Scarface big screen beautif...          1
8   real classic. shipload sailors trying get town...          1
9   Serials short subjects originally shown theate...          1
10  strange sex comedy there`s little comedy whole...          0
11  many problems film, worst continuity; re-editi...          0
12  Rosie wasted lot TV time talking Tainos super ...          0
13  Man, people got chill. movie artistic genius. ...       

In [ ]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-bmxjj3ty
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-bmxjj3ty
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11754 sha256=5d4bf57a87cc7e012d6910d8c8a60a8627b12f248404ad652fe47e664c374bfe
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vvmq2gx/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [ ]:
import preprocess_kgptalkie as ps
import re
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [ ]:
df['SentimentText'] = df['SentimentText'].apply(lambda x: get_clean(x))

In [ ]:
df.head()

,SentimentText,Sentiment
0,first think another disney movie might good it...,1
1,put aside dr house repeat missed desperate hou...,0
2,big fan stephen kings work film made even grea...,1
3,watched horrid thing tv needless say one movie...,0
4,truly enjoyed film acting terrific plot jeff c...,1


In [ ]:
tfidf = TfidfVectorizer(max_features=1000)
X = df['SentimentText']
y = df['Sentiment']

X = tfidf.fit_transform(X)

In [ ]:
X

<19x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1474 stored elements in Compressed Sparse Row format>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12)

In [ ]:
clf = LinearSVC()
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
y_pred = clf.predict(X_test)


In [ ]:
print (classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

